# Starbucks Capstone Challenge

### Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

### Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import math
import json
from datetime import datetime

%matplotlib inline

### Reading the json files

In [2]:
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

### Exploring json files and analyzing the features.

#### portfolio.json

In [3]:
print("Portfolio Size = {}".format(portfolio.shape))
portfolio.head()

Portfolio Size = (10, 6)


,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


#### profile.json file

In [4]:
print("Profile Size = {}".format(profile.shape))
profile.head()

Profile Size = (17000, 5)


,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


#### transcript.json

In [5]:
print("Transcript Size = {}".format(transcript.shape))
transcript.head()

Transcript Size = (306534, 4)


,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


### Checking for Nan values

In [6]:
def check_nan(dataframe):
    nan_columns = dataframe.columns[dataframe.isna().any()].tolist()
    total = len(dataframe)
    print("Total : {}".format(total))
    for column in nan_columns:
        count = dataframe[column].isna().sum()
        print("{} : {}, remaining : {}".format(column, count, total - count))

In [7]:
check_nan(portfolio)

Total : 10


In [8]:
portfolio_nan = portfolio

In [9]:
check_nan(profile)

Total : 17000
gender : 2175, remaining : 14825
income : 2175, remaining : 14825


In [10]:
profile_nan = profile.dropna()
profile_nan = profile_nan.reset_index(drop=True)
profile_nan

,gender,age,id,became_member_on,income
0,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
1,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
2,M,68,e2127556f4f64592b11af22de27a7932,20180426,70000.0
3,M,65,389bc3fa690240e798340f5a15918d5c,20180209,53000.0
4,M,58,2eeac8d8feae4a8cad5a6af0499a211d,20171111,51000.0
...,...,...,...,...,...
14820,F,45,6d5f3a774f3d4714ab0c092238f3a1d7,20180604,54000.0
14821,M,61,2cb4f97358b841b9a9773a7aa05a9d77,20180713,72000.0
14822,M,49,01d26f638c274aa0b965d24cefe3183f,20170126,73000.0
14823,F,83,9dc1421481194dcd9400aec7c9ae6366,20160307,50000.0


In [11]:
check_nan(transcript)

Total : 306534


In [12]:
transcript_nan = transcript

### One-hot encoding

In [13]:
def do_one_hot_encoding(dataframe, column_list):
    for column in column_list:
        column_dummies = pd.get_dummies(dataframe[column])
        dataframe = dataframe.drop(column, axis = 1)
        dataframe = dataframe.join(column_dummies)
    return dataframe

In [14]:
columns = ['offer_type']
portfolio_encoded = do_one_hot_encoding(portfolio_nan, columns)
portfolio_encoded.head()

,reward,channels,difficulty,duration,id,bogo,discount,informational
0,10,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0
1,10,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0
2,0,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,0,0,1
3,5,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0
4,5,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0


In [15]:
columns = ['gender']
profile_encoded = do_one_hot_encoding(profile_nan, columns)
profile_encoded.head()

,age,id,became_member_on,income,F,M,O
0,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0,1,0,0
1,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,1,0,0
2,68,e2127556f4f64592b11af22de27a7932,20180426,70000.0,0,1,0
3,65,389bc3fa690240e798340f5a15918d5c,20180209,53000.0,0,1,0
4,58,2eeac8d8feae4a8cad5a6af0499a211d,20171111,51000.0,0,1,0


In [16]:
columns = ['event']
transcript_encoded = do_one_hot_encoding(transcript_nan, columns)
transcript_encoded.head()

,person,value,time,offer completed,offer received,offer viewed,transaction
0,78afa995795e4d85b5d9ceeca43f5fef,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,0,1,0,0
1,a03223e636434f42ac4c3df47e8bac43,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,0,1,0,0
2,e2127556f4f64592b11af22de27a7932,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,0,1,0,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,0,1,0,0
4,68617ca6246f4fbc85e91a2a49552598,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,0,1,0,0


In [17]:
all_keys  = []
def display_all_keys(x):
   
    if x.keys() not in all_keys:
        all_keys.append(x.keys())
        

In [18]:
transcript_encoded['value'].apply(lambda x: display_all_keys(x))
print(all_keys)

[dict_keys(['offer id']), dict_keys(['amount']), dict_keys(['offer_id', 'reward'])]


In [19]:
def change_name(x, from_name, to_name):
    if from_name in x.keys():
        x[to_name] = x.pop(from_name)
    return x

In [20]:
transcript_encoded['value'].apply(lambda x: change_name(x, 'offer id', 'offer_id'))

0         {'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1         {'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2         {'offer_id': '2906b810c7d4411798c6938adc9daaa5'}
3         {'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4         {'offer_id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}
                                ...                       
306529                      {'amount': 1.5899999999999999}
306530                                    {'amount': 9.53}
306531                                    {'amount': 3.61}
306532                      {'amount': 3.5300000000000002}
306533                                    {'amount': 4.05}
Name: value, Length: 306534, dtype: object

In [21]:
all_keys = []
transcript_encoded['value'].apply(lambda x: display_all_keys(x))
print(all_keys)

[dict_keys(['offer_id']), dict_keys(['amount']), dict_keys(['offer_id', 'reward'])]


In [22]:
def check_not_offer(x, column):
    if column in x.keys() :
        return x[column]

In [23]:
offer_ids = transcript_encoded['value'].apply(lambda x: check_not_offer(x, 'offer_id'))

In [24]:
offer_ids = offer_ids.to_frame()
offer_ids.columns = ['offer_id']
offer_ids.head()

,offer_id
0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,2906b810c7d4411798c6938adc9daaa5
3,fafdcd668e3743c1bb461111dcafc2a4
4,4d5c57ea9a6940dd891ad53e9dbe8da0


In [25]:
transcript_offer_id = transcript_encoded.join(offer_ids)

In [26]:
transcript_offer_id

,person,value,time,offer completed,offer received,offer viewed,transaction,offer_id
0,78afa995795e4d85b5d9ceeca43f5fef,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,a03223e636434f42ac4c3df47e8bac43,{'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,e2127556f4f64592b11af22de27a7932,{'offer_id': '2906b810c7d4411798c6938adc9daaa5'},0,0,1,0,0,2906b810c7d4411798c6938adc9daaa5
3,8ec6ce2a7e7949b1bf142def7d0e0586,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4
4,68617ca6246f4fbc85e91a2a49552598,{'offer_id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0
...,...,...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,{'amount': 1.5899999999999999},714,0,0,0,1,None
306530,68213b08d99a4ae1b0dcb72aebd9aa35,{'amount': 9.53},714,0,0,0,1,None
306531,a00058cf10334a308c68e7631c529907,{'amount': 3.61},714,0,0,0,1,None
306532,76ddbd6576844afe811f1a3c0fbb5bec,{'amount': 3.5300000000000002},714,0,0,0,1,None


In [27]:
column = 'amount'
offer_ids = transcript_encoded['value'].apply(lambda x: check_not_offer(x, column))
offer_ids = offer_ids.to_frame()
offer_ids.columns = [column]
offer_ids

,amount
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
306529,1.59
306530,9.53
306531,3.61
306532,3.53


In [28]:
transcript_amount = transcript_offer_id.join(offer_ids)
transcript_amount.head()

,person,value,time,offer completed,offer received,offer viewed,transaction,offer_id,amount
0,78afa995795e4d85b5d9ceeca43f5fef,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN
1,a03223e636434f42ac4c3df47e8bac43,{'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN
2,e2127556f4f64592b11af22de27a7932,{'offer_id': '2906b810c7d4411798c6938adc9daaa5'},0,0,1,0,0,2906b810c7d4411798c6938adc9daaa5,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4,NaN
4,68617ca6246f4fbc85e91a2a49552598,{'offer_id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN


In [29]:
column = 'reward'
offer_ids = transcript_encoded['value'].apply(lambda x: check_not_offer(x, column))
offer_ids = offer_ids.to_frame()
offer_ids.columns = [column]
offer_ids.dropna()

,reward
12658,2.0
12672,2.0
12679,5.0
12692,10.0
12697,10.0
...,...
306475,3.0
306497,3.0
306506,2.0
306509,2.0


In [30]:
transcript_reward = transcript_amount.join(offer_ids)
transcript_reward.head()

,person,value,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward
0,78afa995795e4d85b5d9ceeca43f5fef,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
1,a03223e636434f42ac4c3df47e8bac43,{'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN
2,e2127556f4f64592b11af22de27a7932,{'offer_id': '2906b810c7d4411798c6938adc9daaa5'},0,0,1,0,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN
4,68617ca6246f4fbc85e91a2a49552598,{'offer_id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN


In [31]:
transcript_on_value = transcript_reward.drop('value', axis = 1)
transcript_on_value

,person,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
1,a03223e636434f42ac4c3df47e8bac43,0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN
2,e2127556f4f64592b11af22de27a7932,0,0,1,0,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN
4,68617ca6246f4fbc85e91a2a49552598,0,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,714,0,0,0,1,None,1.59,NaN
306530,68213b08d99a4ae1b0dcb72aebd9aa35,714,0,0,0,1,None,9.53,NaN
306531,a00058cf10334a308c68e7631c529907,714,0,0,0,1,None,3.61,NaN
306532,76ddbd6576844afe811f1a3c0fbb5bec,714,0,0,0,1,None,3.53,NaN


In [32]:
portfolio_encoded

,reward,channels,difficulty,duration,id,bogo,discount,informational
0,10,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0
1,10,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0
2,0,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,0,0,1
3,5,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0
4,5,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0
5,3,"[web, email, mobile, social]",7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,0,1,0
6,2,"[web, email, mobile, social]",10,10,fafdcd668e3743c1bb461111dcafc2a4,0,1,0
7,0,"[email, mobile, social]",0,3,5a8bc65990b245e5a138643cd4eb9837,0,0,1
8,5,"[web, email, mobile, social]",5,5,f19421c1d4aa40978ebb69ca19b0e20d,1,0,0
9,2,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,0,1,0


In [33]:
def list_to_dataframe(x, column):
    if column in x:
        return 1
    else:
        return 0

In [34]:
values = ['email', 'web', 'mobile', 'social']
for i in values:
    portfolio_encoded_series = portfolio_encoded['channels'].apply(lambda x: list_to_dataframe(x, i))
    portfolio_encoded_series = portfolio_encoded_series.to_frame()
    portfolio_encoded_series.columns = [i]
    portfolio_encoded = portfolio_encoded.join(portfolio_encoded_series)
portfolio_encoded

,reward,channels,difficulty,duration,id,bogo,discount,informational,email,web,mobile,social
0,10,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0,1,0,1,1
1,10,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0,1,1,1,1
2,0,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,0,0,1,1,1,1,0
3,5,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0,1,1,1,0
4,5,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0,1,1,0,0
5,3,"[web, email, mobile, social]",7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,0,1,0,1,1,1,1
6,2,"[web, email, mobile, social]",10,10,fafdcd668e3743c1bb461111dcafc2a4,0,1,0,1,1,1,1
7,0,"[email, mobile, social]",0,3,5a8bc65990b245e5a138643cd4eb9837,0,0,1,1,0,1,1
8,5,"[web, email, mobile, social]",5,5,f19421c1d4aa40978ebb69ca19b0e20d,1,0,0,1,1,1,1
9,2,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,0,1,0,1,1,1,0


In [35]:
portfolio_channels = portfolio_encoded.drop('channels', axis = 1)
portfolio_channels

,reward,difficulty,duration,id,bogo,discount,informational,email,web,mobile,social
0,10,10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0,1,0,1,1
1,10,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0,1,1,1,1
2,0,0,4,3f207df678b143eea3cee63160fa8bed,0,0,1,1,1,1,0
3,5,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0,1,1,1,0
4,5,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0,1,1,0,0
5,3,7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,0,1,0,1,1,1,1
6,2,10,10,fafdcd668e3743c1bb461111dcafc2a4,0,1,0,1,1,1,1
7,0,0,3,5a8bc65990b245e5a138643cd4eb9837,0,0,1,1,0,1,1
8,5,5,5,f19421c1d4aa40978ebb69ca19b0e20d,1,0,0,1,1,1,1
9,2,10,7,2906b810c7d4411798c6938adc9daaa5,0,1,0,1,1,1,0


In [36]:
#if 'email' is same in every row then its redundant
portfolio_channels = portfolio_channels.drop('email', axis = 1)
portfolio_channels

,reward,difficulty,duration,id,bogo,discount,informational,web,mobile,social
0,10,10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0,0,1,1
1,10,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0,1,1,1
2,0,0,4,3f207df678b143eea3cee63160fa8bed,0,0,1,1,1,0
3,5,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0,1,1,0
4,5,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0,1,0,0
5,3,7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,0,1,0,1,1,1
6,2,10,10,fafdcd668e3743c1bb461111dcafc2a4,0,1,0,1,1,1
7,0,0,3,5a8bc65990b245e5a138643cd4eb9837,0,0,1,0,1,1
8,5,5,5,f19421c1d4aa40978ebb69ca19b0e20d,1,0,0,1,1,1
9,2,10,7,2906b810c7d4411798c6938adc9daaa5,0,1,0,1,1,0


### After encoding

In [37]:
portfolio_channels

,reward,difficulty,duration,id,bogo,discount,informational,web,mobile,social
0,10,10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0,0,1,1
1,10,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0,1,1,1
2,0,0,4,3f207df678b143eea3cee63160fa8bed,0,0,1,1,1,0
3,5,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0,1,1,0
4,5,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0,1,0,0
5,3,7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,0,1,0,1,1,1
6,2,10,10,fafdcd668e3743c1bb461111dcafc2a4,0,1,0,1,1,1
7,0,0,3,5a8bc65990b245e5a138643cd4eb9837,0,0,1,0,1,1
8,5,5,5,f19421c1d4aa40978ebb69ca19b0e20d,1,0,0,1,1,1
9,2,10,7,2906b810c7d4411798c6938adc9daaa5,0,1,0,1,1,0


In [38]:
transcript_on_value

,person,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
1,a03223e636434f42ac4c3df47e8bac43,0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN
2,e2127556f4f64592b11af22de27a7932,0,0,1,0,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN
4,68617ca6246f4fbc85e91a2a49552598,0,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,714,0,0,0,1,None,1.59,NaN
306530,68213b08d99a4ae1b0dcb72aebd9aa35,714,0,0,0,1,None,9.53,NaN
306531,a00058cf10334a308c68e7631c529907,714,0,0,0,1,None,3.61,NaN
306532,76ddbd6576844afe811f1a3c0fbb5bec,714,0,0,0,1,None,3.53,NaN


In [39]:
profile_encoded

,age,id,became_member_on,income,F,M,O
0,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0,1,0,0
1,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,1,0,0
2,68,e2127556f4f64592b11af22de27a7932,20180426,70000.0,0,1,0
3,65,389bc3fa690240e798340f5a15918d5c,20180209,53000.0,0,1,0
4,58,2eeac8d8feae4a8cad5a6af0499a211d,20171111,51000.0,0,1,0
...,...,...,...,...,...,...,...
14820,45,6d5f3a774f3d4714ab0c092238f3a1d7,20180604,54000.0,1,0,0
14821,61,2cb4f97358b841b9a9773a7aa05a9d77,20180713,72000.0,0,1,0
14822,49,01d26f638c274aa0b965d24cefe3183f,20170126,73000.0,0,1,0
14823,83,9dc1421481194dcd9400aec7c9ae6366,20160307,50000.0,1,0,0


In [40]:
transcript_on_value['reward'].fillna(0, inplace=True)
transcript_on_value

,person,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,0.0
1,a03223e636434f42ac4c3df47e8bac43,0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,0.0
2,e2127556f4f64592b11af22de27a7932,0,0,1,0,0,2906b810c7d4411798c6938adc9daaa5,NaN,0.0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,0.0
4,68617ca6246f4fbc85e91a2a49552598,0,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,0.0
...,...,...,...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,714,0,0,0,1,None,1.59,0.0
306530,68213b08d99a4ae1b0dcb72aebd9aa35,714,0,0,0,1,None,9.53,0.0
306531,a00058cf10334a308c68e7631c529907,714,0,0,0,1,None,3.61,0.0
306532,76ddbd6576844afe811f1a3c0fbb5bec,714,0,0,0,1,None,3.53,0.0


In [41]:
transcript_on_value['amount'].fillna(0, inplace=True)
transcript_on_value

,person,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.00,0.0
1,a03223e636434f42ac4c3df47e8bac43,0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.00,0.0
2,e2127556f4f64592b11af22de27a7932,0,0,1,0,0,2906b810c7d4411798c6938adc9daaa5,0.00,0.0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4,0.00,0.0
4,68617ca6246f4fbc85e91a2a49552598,0,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.00,0.0
...,...,...,...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,714,0,0,0,1,None,1.59,0.0
306530,68213b08d99a4ae1b0dcb72aebd9aa35,714,0,0,0,1,None,9.53,0.0
306531,a00058cf10334a308c68e7631c529907,714,0,0,0,1,None,3.61,0.0
306532,76ddbd6576844afe811f1a3c0fbb5bec,714,0,0,0,1,None,3.53,0.0


In [42]:
p_columns = portfolio_channels.columns
p_columns = p_columns.to_list()

In [43]:
for n, i in enumerate(p_columns):
    if i!='offer_id':
        p_columns[n] = 'offer_' + p_columns[n]
p_columns

['offer_reward',
 'offer_difficulty',
 'offer_duration',
 'offer_id',
 'offer_bogo',
 'offer_discount',
 'offer_informational',
 'offer_web',
 'offer_mobile',
 'offer_social']

In [44]:
portfolio_channels.columns = p_columns
portfolio_channels

,offer_reward,offer_difficulty,offer_duration,offer_id,offer_bogo,offer_discount,offer_informational,offer_web,offer_mobile,offer_social
0,10,10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0,0,1,1
1,10,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0,1,1,1
2,0,0,4,3f207df678b143eea3cee63160fa8bed,0,0,1,1,1,0
3,5,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0,1,1,0
4,5,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0,1,0,0
5,3,7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,0,1,0,1,1,1
6,2,10,10,fafdcd668e3743c1bb461111dcafc2a4,0,1,0,1,1,1
7,0,0,3,5a8bc65990b245e5a138643cd4eb9837,0,0,1,0,1,1
8,5,5,5,f19421c1d4aa40978ebb69ca19b0e20d,1,0,0,1,1,1
9,2,10,7,2906b810c7d4411798c6938adc9daaa5,0,1,0,1,1,0


In [51]:
portfolio_channels.to_csv("portfolio_formatted.csv", index=False)

In [52]:
transcript_on_value.to_csv("transcript_formatted.csv", index=False)

In [53]:
profile_encoded.to_csv("profile_formatted.csv", index=False)

In [54]:
portfolio_new = pd.read_csv("portfolio_formatted.csv")
transcript_new = pd.read_csv("transcript_formatted.csv")
profile_new = pd.read_csv("profile_formatted.csv")

In [55]:
portfolio_new

,offer_reward,offer_difficulty,offer_duration,offer_id,offer_bogo,offer_discount,offer_informational,offer_web,offer_mobile,offer_social
0,10,10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0,0,1,1
1,10,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0,1,1,1
2,0,0,4,3f207df678b143eea3cee63160fa8bed,0,0,1,1,1,0
3,5,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0,1,1,0
4,5,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0,1,0,0
5,3,7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,0,1,0,1,1,1
6,2,10,10,fafdcd668e3743c1bb461111dcafc2a4,0,1,0,1,1,1
7,0,0,3,5a8bc65990b245e5a138643cd4eb9837,0,0,1,0,1,1
8,5,5,5,f19421c1d4aa40978ebb69ca19b0e20d,1,0,0,1,1,1
9,2,10,7,2906b810c7d4411798c6938adc9daaa5,0,1,0,1,1,0


In [56]:
transcript_new

,person,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.00,0.0
1,a03223e636434f42ac4c3df47e8bac43,0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.00,0.0
2,e2127556f4f64592b11af22de27a7932,0,0,1,0,0,2906b810c7d4411798c6938adc9daaa5,0.00,0.0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4,0.00,0.0
4,68617ca6246f4fbc85e91a2a49552598,0,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.00,0.0
...,...,...,...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,714,0,0,0,1,NaN,1.59,0.0
306530,68213b08d99a4ae1b0dcb72aebd9aa35,714,0,0,0,1,NaN,9.53,0.0
306531,a00058cf10334a308c68e7631c529907,714,0,0,0,1,NaN,3.61,0.0
306532,76ddbd6576844afe811f1a3c0fbb5bec,714,0,0,0,1,NaN,3.53,0.0


In [57]:
profile_new

,age,id,became_member_on,income,F,M,O
0,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0,1,0,0
1,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,1,0,0
2,68,e2127556f4f64592b11af22de27a7932,20180426,70000.0,0,1,0
3,65,389bc3fa690240e798340f5a15918d5c,20180209,53000.0,0,1,0
4,58,2eeac8d8feae4a8cad5a6af0499a211d,20171111,51000.0,0,1,0
...,...,...,...,...,...,...,...
14820,45,6d5f3a774f3d4714ab0c092238f3a1d7,20180604,54000.0,1,0,0
14821,61,2cb4f97358b841b9a9773a7aa05a9d77,20180713,72000.0,0,1,0
14822,49,01d26f638c274aa0b965d24cefe3183f,20170126,73000.0,0,1,0
14823,83,9dc1421481194dcd9400aec7c9ae6366,20160307,50000.0,1,0,0


In [2]:
profile = pd.read_csv("profile_formatted.csv")
portfolio = pd.read_csv("portfolio_formatted.csv")
transcript = pd.read_csv("transcript_formatted.csv")

In [3]:
profile

,age,id,became_member_on,income,F,M,O
0,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0,1,0,0
1,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,1,0,0
2,68,e2127556f4f64592b11af22de27a7932,20180426,70000.0,0,1,0
3,65,389bc3fa690240e798340f5a15918d5c,20180209,53000.0,0,1,0
4,58,2eeac8d8feae4a8cad5a6af0499a211d,20171111,51000.0,0,1,0
...,...,...,...,...,...,...,...
14820,45,6d5f3a774f3d4714ab0c092238f3a1d7,20180604,54000.0,1,0,0
14821,61,2cb4f97358b841b9a9773a7aa05a9d77,20180713,72000.0,0,1,0
14822,49,01d26f638c274aa0b965d24cefe3183f,20170126,73000.0,0,1,0
14823,83,9dc1421481194dcd9400aec7c9ae6366,20160307,50000.0,1,0,0


In [4]:
portfolio

,offer_reward,offer_difficulty,offer_duration,offer_id,offer_bogo,offer_discount,offer_informational,offer_web,offer_mobile,offer_social
0,10,10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0,0,1,1
1,10,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0,1,1,1
2,0,0,4,3f207df678b143eea3cee63160fa8bed,0,0,1,1,1,0
3,5,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0,1,1,0
4,5,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0,1,0,0
5,3,7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,0,1,0,1,1,1
6,2,10,10,fafdcd668e3743c1bb461111dcafc2a4,0,1,0,1,1,1
7,0,0,3,5a8bc65990b245e5a138643cd4eb9837,0,0,1,0,1,1
8,5,5,5,f19421c1d4aa40978ebb69ca19b0e20d,1,0,0,1,1,1
9,2,10,7,2906b810c7d4411798c6938adc9daaa5,0,1,0,1,1,0


In [5]:
transcript

,person,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.00,0.0
1,a03223e636434f42ac4c3df47e8bac43,0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.00,0.0
2,e2127556f4f64592b11af22de27a7932,0,0,1,0,0,2906b810c7d4411798c6938adc9daaa5,0.00,0.0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4,0.00,0.0
4,68617ca6246f4fbc85e91a2a49552598,0,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.00,0.0
...,...,...,...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,714,0,0,0,1,NaN,1.59,0.0
306530,68213b08d99a4ae1b0dcb72aebd9aa35,714,0,0,0,1,NaN,9.53,0.0
306531,a00058cf10334a308c68e7631c529907,714,0,0,0,1,NaN,3.61,0.0
306532,76ddbd6576844afe811f1a3c0fbb5bec,714,0,0,0,1,NaN,3.53,0.0


In [15]:
profile['became_member_on'] = profile['became_member_on'].apply(lambda x: convert_to_datetime(x))

0       2017-07-15
1       2017-05-09
2       2018-04-26
3       2018-02-09
4       2017-11-11
           ...    
14820   2018-06-04
14821   2018-07-13
14822   2017-01-26
14823   2016-03-07
14824   2017-07-22
Name: became_member_on, Length: 14825, dtype: datetime64[ns]

In [13]:
def convert_to_datetime(x):
    value = datetime.strptime(str(x), '%Y%m%d')
    return value

In [16]:
profile_became_member_on = profile

In [18]:
profile_became_member_on['became_member_on_year'] = profile_became_member_on['became_member_on'].apply(lambda x: x.year)
profile_became_member_on

,age,id,became_member_on,income,F,M,O,became_member_on_year
0,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0,1,0,0,2017
1,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0,1,0,0,2017
2,68,e2127556f4f64592b11af22de27a7932,2018-04-26,70000.0,0,1,0,2018
3,65,389bc3fa690240e798340f5a15918d5c,2018-02-09,53000.0,0,1,0,2018
4,58,2eeac8d8feae4a8cad5a6af0499a211d,2017-11-11,51000.0,0,1,0,2017
...,...,...,...,...,...,...,...,...
14820,45,6d5f3a774f3d4714ab0c092238f3a1d7,2018-06-04,54000.0,1,0,0,2018
14821,61,2cb4f97358b841b9a9773a7aa05a9d77,2018-07-13,72000.0,0,1,0,2018
14822,49,01d26f638c274aa0b965d24cefe3183f,2017-01-26,73000.0,0,1,0,2017
14823,83,9dc1421481194dcd9400aec7c9ae6366,2016-03-07,50000.0,1,0,0,2016


In [20]:
profile_became_member_on['became_member_on_month'] = profile_became_member_on['became_member_on'].apply(lambda x: x.month)
profile_became_member_on

,age,id,became_member_on,income,F,M,O,became_member_on_year,became_member_on_month
0,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0,1,0,0,2017,7
1,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0,1,0,0,2017,5
2,68,e2127556f4f64592b11af22de27a7932,2018-04-26,70000.0,0,1,0,2018,4
3,65,389bc3fa690240e798340f5a15918d5c,2018-02-09,53000.0,0,1,0,2018,2
4,58,2eeac8d8feae4a8cad5a6af0499a211d,2017-11-11,51000.0,0,1,0,2017,11
...,...,...,...,...,...,...,...,...,...
14820,45,6d5f3a774f3d4714ab0c092238f3a1d7,2018-06-04,54000.0,1,0,0,2018,6
14821,61,2cb4f97358b841b9a9773a7aa05a9d77,2018-07-13,72000.0,0,1,0,2018,7
14822,49,01d26f638c274aa0b965d24cefe3183f,2017-01-26,73000.0,0,1,0,2017,1
14823,83,9dc1421481194dcd9400aec7c9ae6366,2016-03-07,50000.0,1,0,0,2016,3


In [22]:
profile_became_member_on = profile_became_member_on.drop('became_member_on', axis = 1)
profile_became_member_on

,age,id,income,F,M,O,became_member_on_year,became_member_on_month
0,55,0610b486422d4921ae7d2bf64640c50b,112000.0,1,0,0,2017,7
1,75,78afa995795e4d85b5d9ceeca43f5fef,100000.0,1,0,0,2017,5
2,68,e2127556f4f64592b11af22de27a7932,70000.0,0,1,0,2018,4
3,65,389bc3fa690240e798340f5a15918d5c,53000.0,0,1,0,2018,2
4,58,2eeac8d8feae4a8cad5a6af0499a211d,51000.0,0,1,0,2017,11
...,...,...,...,...,...,...,...,...
14820,45,6d5f3a774f3d4714ab0c092238f3a1d7,54000.0,1,0,0,2018,6
14821,61,2cb4f97358b841b9a9773a7aa05a9d77,72000.0,0,1,0,2018,7
14822,49,01d26f638c274aa0b965d24cefe3183f,73000.0,0,1,0,2017,1
14823,83,9dc1421481194dcd9400aec7c9ae6366,50000.0,1,0,0,2016,3


In [23]:
column = 'became_member_on_year'
column_dummies = pd.get_dummies(profile_became_member_on[column])
profile_became_member_on = profile_became_member_on.drop(column, axis = 1)
profile_became_member_on = profile_became_member_on.join(column_dummies)
profile_became_member_on

,age,id,income,F,M,O,became_member_on_month,2013,2014,2015,2016,2017,2018
0,55,0610b486422d4921ae7d2bf64640c50b,112000.0,1,0,0,7,0,0,0,0,1,0
1,75,78afa995795e4d85b5d9ceeca43f5fef,100000.0,1,0,0,5,0,0,0,0,1,0
2,68,e2127556f4f64592b11af22de27a7932,70000.0,0,1,0,4,0,0,0,0,0,1
3,65,389bc3fa690240e798340f5a15918d5c,53000.0,0,1,0,2,0,0,0,0,0,1
4,58,2eeac8d8feae4a8cad5a6af0499a211d,51000.0,0,1,0,11,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14820,45,6d5f3a774f3d4714ab0c092238f3a1d7,54000.0,1,0,0,6,0,0,0,0,0,1
14821,61,2cb4f97358b841b9a9773a7aa05a9d77,72000.0,0,1,0,7,0,0,0,0,0,1
14822,49,01d26f638c274aa0b965d24cefe3183f,73000.0,0,1,0,1,0,0,0,0,1,0
14823,83,9dc1421481194dcd9400aec7c9ae6366,50000.0,1,0,0,3,0,0,0,1,0,0


In [24]:
profile_became_member_on['income'].describe()

count     14825.000000
mean      65404.991568
std       21598.299410
min       30000.000000
25%       49000.000000
50%       64000.000000
75%       80000.000000
max      120000.000000
Name: income, dtype: float64

In [29]:
profile_became_member_on['income'] = profile_became_member_on['income'] / 120000.000000
profile_became_member_on

,age,id,income,F,M,O,became_member_on_month,2013,2014,2015,2016,2017,2018
0,55,0610b486422d4921ae7d2bf64640c50b,0.933333,1,0,0,7,0,0,0,0,1,0
1,75,78afa995795e4d85b5d9ceeca43f5fef,0.833333,1,0,0,5,0,0,0,0,1,0
2,68,e2127556f4f64592b11af22de27a7932,0.583333,0,1,0,4,0,0,0,0,0,1
3,65,389bc3fa690240e798340f5a15918d5c,0.441667,0,1,0,2,0,0,0,0,0,1
4,58,2eeac8d8feae4a8cad5a6af0499a211d,0.425000,0,1,0,11,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14820,45,6d5f3a774f3d4714ab0c092238f3a1d7,0.450000,1,0,0,6,0,0,0,0,0,1
14821,61,2cb4f97358b841b9a9773a7aa05a9d77,0.600000,0,1,0,7,0,0,0,0,0,1
14822,49,01d26f638c274aa0b965d24cefe3183f,0.608333,0,1,0,1,0,0,0,0,1,0
14823,83,9dc1421481194dcd9400aec7c9ae6366,0.416667,1,0,0,3,0,0,0,1,0,0


In [30]:
profile_became_member_on['age'].describe()

count    14825.000000
mean        54.393524
std         17.383705
min         18.000000
25%         42.000000
50%         55.000000
75%         66.000000
max        101.000000
Name: age, dtype: float64

In [31]:
profile_became_member_on['age'] / 101

0        0.544554
1        0.742574
2        0.673267
3        0.643564
4        0.574257
           ...   
14820    0.445545
14821    0.603960
14822    0.485149
14823    0.821782
14824    0.613861
Name: age, Length: 14825, dtype: float64

In [32]:
profile_became_member_on['age'] = profile_became_member_on['age'] / 101
profile_became_member_on

,age,id,income,F,M,O,became_member_on_month,2013,2014,2015,2016,2017,2018
0,0.544554,0610b486422d4921ae7d2bf64640c50b,0.933333,1,0,0,7,0,0,0,0,1,0
1,0.742574,78afa995795e4d85b5d9ceeca43f5fef,0.833333,1,0,0,5,0,0,0,0,1,0
2,0.673267,e2127556f4f64592b11af22de27a7932,0.583333,0,1,0,4,0,0,0,0,0,1
3,0.643564,389bc3fa690240e798340f5a15918d5c,0.441667,0,1,0,2,0,0,0,0,0,1
4,0.574257,2eeac8d8feae4a8cad5a6af0499a211d,0.425000,0,1,0,11,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14820,0.445545,6d5f3a774f3d4714ab0c092238f3a1d7,0.450000,1,0,0,6,0,0,0,0,0,1
14821,0.603960,2cb4f97358b841b9a9773a7aa05a9d77,0.600000,0,1,0,7,0,0,0,0,0,1
14822,0.485149,01d26f638c274aa0b965d24cefe3183f,0.608333,0,1,0,1,0,0,0,0,1,0
14823,0.821782,9dc1421481194dcd9400aec7c9ae6366,0.416667,1,0,0,3,0,0,0,1,0,0


In [33]:
transcript

,person,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.00,0.0
1,a03223e636434f42ac4c3df47e8bac43,0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.00,0.0
2,e2127556f4f64592b11af22de27a7932,0,0,1,0,0,2906b810c7d4411798c6938adc9daaa5,0.00,0.0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4,0.00,0.0
4,68617ca6246f4fbc85e91a2a49552598,0,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.00,0.0
...,...,...,...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,714,0,0,0,1,NaN,1.59,0.0
306530,68213b08d99a4ae1b0dcb72aebd9aa35,714,0,0,0,1,NaN,9.53,0.0
306531,a00058cf10334a308c68e7631c529907,714,0,0,0,1,NaN,3.61,0.0
306532,76ddbd6576844afe811f1a3c0fbb5bec,714,0,0,0,1,NaN,3.53,0.0


In [36]:
profile_became_member_on[profile_became_member_on['id'] == '78afa995795e4d85b5d9ceeca43f5fef']

,age,id,income,F,M,O,became_member_on_month,2013,2014,2015,2016,2017,2018
1,0.742574,78afa995795e4d85b5d9ceeca43f5fef,0.833333,1,0,0,5,0,0,0,0,1,0


In [37]:
transcript[transcript['person'] == '0610b486422d4921ae7d2bf64640c50b']

,person,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward
20282,0610b486422d4921ae7d2bf64640c50b,18,0,0,0,1,NaN,21.51,0.0
49501,0610b486422d4921ae7d2bf64640c50b,144,0,0,0,1,NaN,32.28,0.0
150597,0610b486422d4921ae7d2bf64640c50b,408,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.00,0.0
201571,0610b486422d4921ae7d2bf64640c50b,504,0,1,0,0,3f207df678b143eea3cee63160fa8bed,0.00,0.0
227842,0610b486422d4921ae7d2bf64640c50b,528,0,0,0,1,NaN,23.22,0.0
227843,0610b486422d4921ae7d2bf64640c50b,528,1,0,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.00,5.0


In [40]:
profile_became_member_on.columns

Index([                   'age',                     'id',
                       'income',                      'F',
                            'M',                      'O',
       'became_member_on_month',                     2013,
                           2014,                     2015,
                           2016,                     2017,
                           2018],
      dtype='object')

In [47]:
transcript_profile = pd.merge(transcript, profile_became_member_on, left_on = ['person'], right_on = ['id'])
transcript_profile

,person,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward,age,...,F,M,O,became_member_on_month,2013,2014,2015,2016,2017,2018
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.00,0.0,0.742574,...,1,0,0,5,0,0,0,0,1,0
1,78afa995795e4d85b5d9ceeca43f5fef,6,0,0,1,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.00,0.0,0.742574,...,1,0,0,5,0,0,0,0,1,0
2,78afa995795e4d85b5d9ceeca43f5fef,132,0,0,0,1,NaN,19.89,0.0,0.742574,...,1,0,0,5,0,0,0,0,1,0
3,78afa995795e4d85b5d9ceeca43f5fef,132,1,0,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.00,5.0,0.742574,...,1,0,0,5,0,0,0,0,1,0
4,78afa995795e4d85b5d9ceeca43f5fef,144,0,0,0,1,NaN,17.78,0.0,0.742574,...,1,0,0,5,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272757,9fcbff4f8d7241faa4ab8a9d19c8a812,504,0,0,1,0,3f207df678b143eea3cee63160fa8bed,0.00,0.0,0.465347,...,0,1,0,10,0,0,0,0,1,0
272758,9fcbff4f8d7241faa4ab8a9d19c8a812,576,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.00,0.0,0.465347,...,0,1,0,10,0,0,0,0,1,0
272759,9fcbff4f8d7241faa4ab8a9d19c8a812,576,0,0,1,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.00,0.0,0.465347,...,0,1,0,10,0,0,0,0,1,0
272760,3045af4e98794a04a5542d3eac939b1f,576,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.00,0.0,0.574257,...,1,0,0,10,0,0,0,1,0,0


In [46]:
profile_became_member_on[profile_became_member_on['id'] == '78afa995795e4d85b5d9ceeca43f5fef']

,age,id,income,F,M,O,became_member_on_month,2013,2014,2015,2016,2017,2018
1,0.742574,78afa995795e4d85b5d9ceeca43f5fef,0.833333,1,0,0,5,0,0,0,0,1,0


In [48]:
portfolio

,offer_reward,offer_difficulty,offer_duration,offer_id,offer_bogo,offer_discount,offer_informational,offer_web,offer_mobile,offer_social
0,10,10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0,0,1,1
1,10,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0,1,1,1
2,0,0,4,3f207df678b143eea3cee63160fa8bed,0,0,1,1,1,0
3,5,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0,1,1,0
4,5,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0,1,0,0
5,3,7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,0,1,0,1,1,1
6,2,10,10,fafdcd668e3743c1bb461111dcafc2a4,0,1,0,1,1,1
7,0,0,3,5a8bc65990b245e5a138643cd4eb9837,0,0,1,0,1,1
8,5,5,5,f19421c1d4aa40978ebb69ca19b0e20d,1,0,0,1,1,1
9,2,10,7,2906b810c7d4411798c6938adc9daaa5,0,1,0,1,1,0


In [49]:
transcript_profile.shape

(272762, 22)

In [51]:
pd.merge(transcript_profile, portfolio, left_on = ['offer_id'], right_on = ['offer_id']).shape

(148805, 31)

In [52]:
272762 - 148805

123957

In [53]:
transcript_profile_portfolio = pd.merge(transcript_profile, portfolio, left_on = ['offer_id'], right_on = ['offer_id'])
transcript_profile_portfolio

,person,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward,age,...,2018,offer_reward,offer_difficulty,offer_duration,offer_bogo,offer_discount,offer_informational,offer_web,offer_mobile,offer_social
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,0.742574,...,0,5,5,7,1,0,0,1,1,0
1,78afa995795e4d85b5d9ceeca43f5fef,6,0,0,1,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,0.742574,...,0,5,5,7,1,0,0,1,1,0
2,78afa995795e4d85b5d9ceeca43f5fef,132,1,0,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,5.0,0.742574,...,0,5,5,7,1,0,0,1,1,0
3,e2127556f4f64592b11af22de27a7932,408,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,0.673267,...,1,5,5,7,1,0,0,1,1,0
4,e2127556f4f64592b11af22de27a7932,420,0,0,1,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,0.673267,...,1,5,5,7,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148800,8578196a074a4f328976e334fa9383a3,504,0,0,1,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.0,0.0,0.475248,...,1,10,10,5,1,0,0,1,1,1
148801,9fcbff4f8d7241faa4ab8a9d19c8a812,576,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.0,0.0,0.465347,...,0,10,10,5,1,0,0,1,1,1
148802,9fcbff4f8d7241faa4ab8a9d19c8a812,576,0,0,1,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.0,0.0,0.465347,...,0,10,10,5,1,0,0,1,1,1
148803,3045af4e98794a04a5542d3eac939b1f,576,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.0,0.0,0.574257,...,0,10,10,5,1,0,0,1,1,1


In [57]:
transcript_profile_portfolio[transcript_profile_portfolio['offer completed']==1].shape

(32444, 31)

In [58]:
transcript_profile_portfolio[transcript_profile_portfolio['offer completed']==0].shape

(116361, 31)

In [59]:
transcript_profile_portfolio.to_csv("transcript_profile_portfolio.csv")

In [4]:
transcript_profile_portfolio = pd.read_csv("transcript_profile_portfolio.csv")
transcript_profile_portfolio = transcript_profile_portfolio.drop('Unnamed: 0', axis=1)
transcript_profile_portfolio.to_csv("transcript_profile_portfolio.csv", index=False)
transcript_profile_portfolio = pd.read_csv("transcript_profile_portfolio.csv")
transcript_profile_portfolio

,person,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward,age,...,2018,offer_reward,offer_difficulty,offer_duration,offer_bogo,offer_discount,offer_informational,offer_web,offer_mobile,offer_social
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,0.742574,...,0,5,5,7,1,0,0,1,1,0
1,78afa995795e4d85b5d9ceeca43f5fef,6,0,0,1,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,0.742574,...,0,5,5,7,1,0,0,1,1,0
2,78afa995795e4d85b5d9ceeca43f5fef,132,1,0,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,5.0,0.742574,...,0,5,5,7,1,0,0,1,1,0
3,e2127556f4f64592b11af22de27a7932,408,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,0.673267,...,1,5,5,7,1,0,0,1,1,0
4,e2127556f4f64592b11af22de27a7932,420,0,0,1,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,0.673267,...,1,5,5,7,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148800,8578196a074a4f328976e334fa9383a3,504,0,0,1,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.0,0.0,0.475248,...,1,10,10,5,1,0,0,1,1,1
148801,9fcbff4f8d7241faa4ab8a9d19c8a812,576,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.0,0.0,0.465347,...,0,10,10,5,1,0,0,1,1,1
148802,9fcbff4f8d7241faa4ab8a9d19c8a812,576,0,0,1,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.0,0.0,0.465347,...,0,10,10,5,1,0,0,1,1,1
148803,3045af4e98794a04a5542d3eac939b1f,576,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.0,0.0,0.574257,...,0,10,10,5,1,0,0,1,1,1


In [5]:
transcript_profile_portfolio = transcript_profile_portfolio.drop('offer_id', axis=1)
transcript_profile_portfolio

,person,time,offer completed,offer received,offer viewed,transaction,amount,reward,age,id,...,2018,offer_reward,offer_difficulty,offer_duration,offer_bogo,offer_discount,offer_informational,offer_web,offer_mobile,offer_social
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,0.0,0.0,0.742574,78afa995795e4d85b5d9ceeca43f5fef,...,0,5,5,7,1,0,0,1,1,0
1,78afa995795e4d85b5d9ceeca43f5fef,6,0,0,1,0,0.0,0.0,0.742574,78afa995795e4d85b5d9ceeca43f5fef,...,0,5,5,7,1,0,0,1,1,0
2,78afa995795e4d85b5d9ceeca43f5fef,132,1,0,0,0,0.0,5.0,0.742574,78afa995795e4d85b5d9ceeca43f5fef,...,0,5,5,7,1,0,0,1,1,0
3,e2127556f4f64592b11af22de27a7932,408,0,1,0,0,0.0,0.0,0.673267,e2127556f4f64592b11af22de27a7932,...,1,5,5,7,1,0,0,1,1,0
4,e2127556f4f64592b11af22de27a7932,420,0,0,1,0,0.0,0.0,0.673267,e2127556f4f64592b11af22de27a7932,...,1,5,5,7,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148800,8578196a074a4f328976e334fa9383a3,504,0,0,1,0,0.0,0.0,0.475248,8578196a074a4f328976e334fa9383a3,...,1,10,10,5,1,0,0,1,1,1
148801,9fcbff4f8d7241faa4ab8a9d19c8a812,576,0,1,0,0,0.0,0.0,0.465347,9fcbff4f8d7241faa4ab8a9d19c8a812,...,0,10,10,5,1,0,0,1,1,1
148802,9fcbff4f8d7241faa4ab8a9d19c8a812,576,0,0,1,0,0.0,0.0,0.465347,9fcbff4f8d7241faa4ab8a9d19c8a812,...,0,10,10,5,1,0,0,1,1,1
148803,3045af4e98794a04a5542d3eac939b1f,576,0,1,0,0,0.0,0.0,0.574257,3045af4e98794a04a5542d3eac939b1f,...,0,10,10,5,1,0,0,1,1,1


In [6]:
transcript_profile_portfolio = transcript_profile_portfolio.drop('id', axis=1)
transcript_profile_portfolio

,person,time,offer completed,offer received,offer viewed,transaction,amount,reward,age,income,...,2018,offer_reward,offer_difficulty,offer_duration,offer_bogo,offer_discount,offer_informational,offer_web,offer_mobile,offer_social
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,0.0,0.0,0.742574,0.833333,...,0,5,5,7,1,0,0,1,1,0
1,78afa995795e4d85b5d9ceeca43f5fef,6,0,0,1,0,0.0,0.0,0.742574,0.833333,...,0,5,5,7,1,0,0,1,1,0
2,78afa995795e4d85b5d9ceeca43f5fef,132,1,0,0,0,0.0,5.0,0.742574,0.833333,...,0,5,5,7,1,0,0,1,1,0
3,e2127556f4f64592b11af22de27a7932,408,0,1,0,0,0.0,0.0,0.673267,0.583333,...,1,5,5,7,1,0,0,1,1,0
4,e2127556f4f64592b11af22de27a7932,420,0,0,1,0,0.0,0.0,0.673267,0.583333,...,1,5,5,7,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148800,8578196a074a4f328976e334fa9383a3,504,0,0,1,0,0.0,0.0,0.475248,0.483333,...,1,10,10,5,1,0,0,1,1,1
148801,9fcbff4f8d7241faa4ab8a9d19c8a812,576,0,1,0,0,0.0,0.0,0.465347,0.783333,...,0,10,10,5,1,0,0,1,1,1
148802,9fcbff4f8d7241faa4ab8a9d19c8a812,576,0,0,1,0,0.0,0.0,0.465347,0.783333,...,0,10,10,5,1,0,0,1,1,1
148803,3045af4e98794a04a5542d3eac939b1f,576,0,1,0,0,0.0,0.0,0.574257,0.650000,...,0,10,10,5,1,0,0,1,1,1


In [7]:
transcript_profile_portfolio.to_csv("transcript_profile_portfolio.csv", index=False)

In [3]:
transcript_profile_portfolio = pd.read_csv("transcript_profile_portfolio.csv")
transcript_profile_portfolio

,person,time,offer completed,offer received,offer viewed,transaction,amount,reward,age,income,...,2018,offer_reward,offer_difficulty,offer_duration,offer_bogo,offer_discount,offer_informational,offer_web,offer_mobile,offer_social
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,0.0,0.0,0.742574,0.833333,...,0,5,5,7,1,0,0,1,1,0
1,78afa995795e4d85b5d9ceeca43f5fef,6,0,0,1,0,0.0,0.0,0.742574,0.833333,...,0,5,5,7,1,0,0,1,1,0
2,78afa995795e4d85b5d9ceeca43f5fef,132,1,0,0,0,0.0,5.0,0.742574,0.833333,...,0,5,5,7,1,0,0,1,1,0
3,e2127556f4f64592b11af22de27a7932,408,0,1,0,0,0.0,0.0,0.673267,0.583333,...,1,5,5,7,1,0,0,1,1,0
4,e2127556f4f64592b11af22de27a7932,420,0,0,1,0,0.0,0.0,0.673267,0.583333,...,1,5,5,7,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148800,8578196a074a4f328976e334fa9383a3,504,0,0,1,0,0.0,0.0,0.475248,0.483333,...,1,10,10,5,1,0,0,1,1,1
148801,9fcbff4f8d7241faa4ab8a9d19c8a812,576,0,1,0,0,0.0,0.0,0.465347,0.783333,...,0,10,10,5,1,0,0,1,1,1
148802,9fcbff4f8d7241faa4ab8a9d19c8a812,576,0,0,1,0,0.0,0.0,0.465347,0.783333,...,0,10,10,5,1,0,0,1,1,1
148803,3045af4e98794a04a5542d3eac939b1f,576,0,1,0,0,0.0,0.0,0.574257,0.650000,...,0,10,10,5,1,0,0,1,1,1


In [5]:
transcript_profile_portfolio = transcript_profile_portfolio.drop('person', axis = 1)

In [7]:
transcript_profile_portfolio.shape

(148805, 28)

In [148]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleNet(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleNet, self).__init__()
        
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.3)
        
        self.sig = nn.Sigmoid()
        
    def forward(self, x):
        out = F.relu(self.fc1(x))
        out = self.dropout(out)
        out = self.fc2(out)
        print("before = ", out)
        out = self.sig(out)
        print("after = ", out)
        return out

In [16]:
transcript_profile_portfolio.shape

(148805, 28)

In [28]:
transcript_profile_portfolio.head()

,time,offer completed,offer received,offer viewed,transaction,amount,reward,age,income,F,...,2018,offer_reward,offer_difficulty,offer_duration,offer_bogo,offer_discount,offer_informational,offer_web,offer_mobile,offer_social
0,0,0,1,0,0,0.0,0.0,0.742574,0.833333,1,...,0,5,5,7,1,0,0,1,1,0
1,6,0,0,1,0,0.0,0.0,0.742574,0.833333,1,...,0,5,5,7,1,0,0,1,1,0
2,132,1,0,0,0,0.0,5.0,0.742574,0.833333,1,...,0,5,5,7,1,0,0,1,1,0
3,408,0,1,0,0,0.0,0.0,0.673267,0.583333,0,...,1,5,5,7,1,0,0,1,1,0
4,420,0,0,1,0,0.0,0.0,0.673267,0.583333,0,...,1,5,5,7,1,0,0,1,1,0


In [40]:
column_name = 'offer completed'
cols = list(transcript_profile_portfolio.columns.values)
cols.pop(cols.index(column_name))
cols.append(column_name)
transcript_profile_portfolio.columns = cols
transcript_profile_portfolio

,time,offer received,offer viewed,transaction,amount,reward,age,income,F,M,...,offer_reward,offer_difficulty,offer_duration,offer_bogo,offer_discount,offer_informational,offer_web,offer_mobile,offer_social,offer completed
0,0,0,1,0,0,0.0,0.0,0.742574,0.833333,1,...,0,5,5,7,1,0,0,1,1,0
1,6,0,0,1,0,0.0,0.0,0.742574,0.833333,1,...,0,5,5,7,1,0,0,1,1,0
2,132,1,0,0,0,0.0,5.0,0.742574,0.833333,1,...,0,5,5,7,1,0,0,1,1,0
3,408,0,1,0,0,0.0,0.0,0.673267,0.583333,0,...,1,5,5,7,1,0,0,1,1,0
4,420,0,0,1,0,0.0,0.0,0.673267,0.583333,0,...,1,5,5,7,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148800,504,0,0,1,0,0.0,0.0,0.475248,0.483333,0,...,1,10,10,5,1,0,0,1,1,1
148801,576,0,1,0,0,0.0,0.0,0.465347,0.783333,0,...,0,10,10,5,1,0,0,1,1,1
148802,576,0,0,1,0,0.0,0.0,0.465347,0.783333,0,...,0,10,10,5,1,0,0,1,1,1
148803,576,0,1,0,0,0.0,0.0,0.574257,0.650000,1,...,0,10,10,5,1,0,0,1,1,1


In [55]:
def split_train_test(dataframe_values, train_percent = 0.7):
    test_size = int(dataframe_values.shape[0] * train_percent)
    np.random.shuffle(dataframe_values)
    train_data = dataframe_values[:test_size]
    test_data = dataframe_values[test_size:]
    print("train size = ", train_data.shape)
    print("test size = ", test_data.shape)
    print("total size = ", test_data.shape[0] + train_data.shape[0])
    return train_data, test_data

In [56]:
train_data, test_data = split_train_test(transcript_profile_portfolio.values)

train size =  (104163, 28)
test size =  (44642, 28)
total size =  148805


In [60]:
train_data[:, :-1].shape

(104163, 27)

In [61]:
train_data[:, -1].shape

(104163,)

In [62]:
def get_loaders(data, batch_size = 32):
    x = torch.from_numpy(data[:, :-1]).float().squeeze()
    y = torch.from_numpy(data[:, -1]).float()
    
    ds = torch.utils.data.TensorDataset(x, y)
    loader = torch.utils.data.DataLoader(ds, batch_size = batch_size)
    return loader

In [63]:
train_loader = get_loaders(train_data)
train_loader

In [64]:
test_loader = get_loaders(test_data)
test_loader

In [161]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
binary_model = SimpleNet(27, 10, 1).to(device)
binary_model

SimpleNet(
  (fc1): Linear(in_features=27, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=1, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (sig): Sigmoid()
)

In [162]:
import torch.optim as optim

optimizer = optim.SGD(binary_model.parameters(), lr = 0.0001, momentum=0.5)
criterion = nn.BCELoss()

In [163]:
def train(model, train_loader, epochs, criterion, optimizer, device):
    
    for epoch in range(0, epochs):
        model.train()
        
        total_loss = 0
        
        for batch in train_loader:
            
            batch_x, batch_y = batch
            
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
            
            optimizer.zero_grad()
            
            y_pred = model(batch_x)
            loss = criterion(y_pred, batch_y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.data.item()
            
        print("Epoch: {}, Loss: {}".format(epoch, total_loss/ len(train_loader)))
        
    return model

In [164]:
trained_model = train(binary_model, train_loader, 10, criterion, optimizer, device)

before =  tensor([[ 9.0528e+00],
        [-2.6150e+01],
        [ 2.9955e+00],
        [ 3.7837e+01],
        [-1.3204e+01],
        [ 2.4190e+00],
        [-1.6832e+01],
        [-1.1235e+01],
        [ 1.4026e+01],
        [-4.3634e+01],
        [ 6.9160e+01],
        [ 1.7737e+01],
        [-1.0291e+01],
        [-7.6590e-01],
        [-1.3788e+01],
        [ 8.2330e-02],
        [ 2.1681e-01],
        [-5.0624e-01],
        [ 1.8407e-02],
        [ 1.1286e+00],
        [ 2.1002e+01],
        [ 2.2086e-02],
        [ 2.0069e+00],
        [-1.0633e+01],
        [-4.3724e+01],
        [-1.0898e+01],
        [ 4.3844e+01],
        [ 5.9785e+01],
        [-7.3891e+00],
        [ 4.7795e-01],
        [ 2.9195e+01],
        [ 8.4399e+00]], device='cuda:0', grad_fn=<AddmmBackward>)
after =  tensor([[9.9988e-01],
        [4.3966e-12],
        [9.5237e-01],
        [1.0000e+00],
        [1.8431e-06],
        [9.1827e-01],
        [4.8981e-08],
        [1.3209e-05],
        [1.0000e+00],
    

        [-2.7432e+00]], device='cuda:0', grad_fn=<AddmmBackward>)
after =  tensor([[1.0000e+00],
        [1.7451e-10],
        [8.7893e-05],
        [4.6813e-01],
        [5.1279e-01],
        [1.0000e+00],
        [1.7016e-05],
        [1.0000e+00],
        [9.9731e-01],
        [1.0000e+00],
        [4.4159e-01],
        [3.9286e-01],
        [1.8622e-02],
        [1.3988e-05],
        [4.3156e-08],
        [5.7207e-09],
        [1.0000e+00],
        [9.9974e-01],
        [5.5697e-04],
        [1.0000e+00],
        [2.9537e-01],
        [9.9965e-01],
        [9.8016e-01],
        [5.9014e-01],
        [1.0000e+00],
        [9.9881e-01],
        [9.9971e-01],
        [3.0010e-02],
        [1.0000e+00],
        [7.9922e-03],
        [1.0000e+00],
        [6.0471e-02]], device='cuda:0', grad_fn=<SigmoidBackward>)
before =  tensor([[ 3.3052e+01],
        [-3.3582e+01],
        [-1.5535e+00],
        [-1.0410e+01],
        [ 7.3147e+00],
        [ 5.8442e+01],
        [ 4.1261e+01],
     

        [ 6.7906e+00]], device='cuda:0', grad_fn=<AddmmBackward>)
after =  tensor([[5.4938e-01],
        [2.4000e-01],
        [1.0000e+00],
        [1.0000e+00],
        [5.0694e-01],
        [6.6113e-05],
        [3.0353e-04],
        [5.3616e-04],
        [2.4739e-03],
        [1.0000e+00],
        [1.0000e+00],
        [1.4217e-07],
        [4.0830e-01],
        [4.4449e-11],
        [1.5600e-02],
        [5.8365e-04],
        [1.0000e+00],
        [9.8214e-01],
        [1.0000e+00],
        [2.0910e-10],
        [1.9477e-21],
        [1.0000e+00],
        [1.0000e+00],
        [8.2800e-03],
        [8.5163e-12],
        [9.9946e-01],
        [4.3808e-01],
        [9.9923e-01],
        [4.6385e-12],
        [9.9994e-01],
        [5.3020e-01],
        [9.9888e-01]], device='cuda:0', grad_fn=<SigmoidBackward>)
before =  tensor([[ 22.3706],
        [-21.8608],
        [  0.3867],
        [-13.5722],
        [ -6.4017],
        [-11.7278],
        [  5.9015],
        [-16.3182],
      

        [ 3.6497]], device='cuda:0', grad_fn=<AddmmBackward>)
after =  tensor([[9.4567e-01],
        [8.4749e-01],
        [9.9969e-01],
        [1.0000e+00],
        [1.0000e+00],
        [9.9918e-01],
        [1.0000e+00],
        [3.9556e-01],
        [1.0000e+00],
        [9.2002e-01],
        [1.0000e+00],
        [7.9419e-01],
        [6.9604e-01],
        [5.8550e-01],
        [1.0000e+00],
        [1.0000e+00],
        [1.0000e+00],
        [1.0000e+00],
        [3.8588e-01],
        [9.7420e-01],
        [5.9121e-01],
        [5.2030e-01],
        [1.0000e+00],
        [1.0000e+00],
        [1.0000e+00],
        [1.0000e+00],
        [9.9953e-01],
        [9.9561e-01],
        [1.0000e+00],
        [6.8595e-01],
        [5.2884e-04],
        [9.7466e-01]], device='cuda:0', grad_fn=<SigmoidBackward>)
before =  tensor([[-7.5632e+00],
        [ 1.4871e+01],
        [-1.0500e+00],
        [ 7.1276e+00],
        [ 1.1953e+00],
        [-6.7631e+00],
        [ 9.8051e+00],
        [

        [1.0000e+00]], device='cuda:0', grad_fn=<SigmoidBackward>)
before =  tensor([[ 3.7816e+01],
        [ 3.4819e+01],
        [ 2.0156e+01],
        [ 4.6540e+00],
        [-2.4944e+00],
        [-7.3545e+00],
        [ 2.9333e+01],
        [-7.9378e-01],
        [ 3.3682e-02],
        [ 4.2586e+00],
        [ 2.3639e+00],
        [-5.5105e+00],
        [ 1.0738e+01],
        [ 9.7613e+00],
        [ 1.9325e+01],
        [ 1.4075e+00],
        [ 3.7576e+01],
        [ 9.9285e+00],
        [ 4.3629e+01],
        [ 1.3721e+01],
        [ 7.3383e+00],
        [ 1.0858e+01],
        [ 2.4200e+01],
        [-4.6555e-01],
        [ 3.1635e+01],
        [ 4.0223e-01],
        [ 2.0068e+01],
        [ 2.0427e+00],
        [ 7.3306e+01],
        [-5.5786e+00],
        [ 1.9470e+01],
        [ 1.8313e+01]], device='cuda:0', grad_fn=<AddmmBackward>)
after =  tensor([[1.0000e+00],
        [1.0000e+00],
        [1.0000e+00],
        [9.9057e-01],
        [7.6251e-02],
        [6.3931e-04],
   

...

        [-0.3047]], device='cuda:0', grad_fn=<AddmmBackward>)
after =  tensor([[0.5475],
        [0.5987],
        [0.6960],
        [0.6160],
        [0.3856],
        [0.5994],
        [0.6009],
        [0.5390],
        [0.5916],
        [0.5412],
        [0.5826],
        [0.5086],
        [0.5351],
        [0.6280],
        [0.5739],
        [0.6493],
        [0.5735],
        [0.5878],
        [0.6097],
        [0.5097],
        [0.5533],
        [0.5743],
        [0.5952],
        [0.4031],
        [0.5851],
        [0.5380],
        [0.5984],
        [0.4061],
        [0.5966],
        [0.6032],
        [0.5344],
        [0.4244]], device='cuda:0', grad_fn=<SigmoidBackward>)
before =  tensor([[ 0.1675],
        [ 0.0795],
        [ 0.3616],
        [ 0.6294],
        [ 0.2774],
        [ 0.7718],
        [-0.0334],
        [ 0.4088],
        [ 0.3441],
        [-0.2436],
        [ 0.2323],
        [-0.2938],
        [ 0.3909],
        [-0.1767],
        [ 0.3605],
        [ 0.6

        [0.6411]], device='cuda:0', grad_fn=<SigmoidBackward>)
before =  tensor([[ 0.2969],
        [ 1.5656],
        [ 0.2591],
        [ 0.5688],
        [ 0.6273],
        [ 0.8557],
        [ 0.5155],
        [ 1.1393],
        [-0.2796],
        [ 0.2759],
        [-0.4317],
        [ 0.2251],
        [ 0.8761],
        [ 0.8552],
        [ 0.5168],
        [ 0.3120],
        [ 0.3910],
        [ 0.8512],
        [ 0.5278],
        [ 0.2844],
        [ 0.3301],
        [ 0.8480],
        [ 0.3225],
        [ 0.5466],
        [ 0.2215],
        [ 0.4725],
        [ 0.0760],
        [ 1.0200],
        [ 0.6244],
        [ 0.3058],
        [ 0.6037],
        [ 0.4137]], device='cuda:0', grad_fn=<AddmmBackward>)
after =  tensor([[0.5737],
        [0.8272],
        [0.5644],
        [0.6385],
        [0.6519],
        [0.7018],
        [0.6261],
        [0.7576],
        [0.4306],
        [0.5685],
        [0.3937],
        [0.5560],
        [0.7060],
        [0.7017],
        [0.6264

        [ 0.3924]], device='cuda:0', grad_fn=<AddmmBackward>)
after =  tensor([[0.5574],
        [0.5981],
        [0.5039],
        [0.3867],
        [0.5329],
        [0.5880],
        [0.5270],
        [0.6315],
        [0.6901],
        [0.5538],
        [0.4636],
        [0.5277],
        [0.6386],
        [0.3086],
        [0.6013],
        [0.5126],
        [0.5942],
        [0.6551],
        [0.5546],
        [0.5557],
        [0.5739],
        [0.5571],
        [0.5566],
        [0.7578],
        [0.4974],
        [0.7052],
        [0.5864],
        [0.5642],
        [0.5087],
        [0.4712],
        [0.5756],
        [0.5969]], device='cuda:0', grad_fn=<SigmoidBackward>)
before =  tensor([[ 0.3835],
        [ 0.9368],
        [ 0.9423],
        [ 0.2286],
        [ 0.2630],
        [ 0.2517],
        [ 0.0268],
        [ 1.1978],
        [ 0.5094],
        [ 0.4789],
        [ 0.5743],
        [ 0.5950],
        [-0.0793],
        [ 0.8255],
        [ 0.3716],
        [ 0.6

        [0.5647]], device='cuda:0', grad_fn=<SigmoidBackward>)
before =  tensor([[ 0.4773],
        [ 0.9401],
        [-0.2673],
        [-0.1509],
        [ 0.8648],
        [ 0.3361],
        [-0.2105],
        [-0.6682],
        [-0.2813],
        [-0.6292],
        [ 0.2127],
        [-0.7495],
        [ 0.2378],
        [-0.3209],
        [-0.4225],
        [-0.9807],
        [-0.7252],
        [ 0.0670],
        [-0.8689],
        [ 0.3682],
        [-0.4173],
        [-0.6239],
        [-0.8164],
        [-0.4597],
        [-0.7983],
        [ 0.7194],
        [ 0.1077],
        [-0.4206],
        [-0.0011],
        [ 0.1287],
        [-0.2079],
        [ 0.8123]], device='cuda:0', grad_fn=<AddmmBackward>)
after =  tensor([[0.6171],
        [0.7191],
        [0.4336],
        [0.4623],
        [0.7037],
        [0.5832],
        [0.4476],
        [0.3389],
        [0.4301],
        [0.3477],
        [0.5530],
        [0.3209],
        [0.5592],
        [0.4205],
        [0.3959

KeyboardInterrupt: 

In [146]:
def test(model, test_loader, criterion, device):
    with torch.no_grad():
        model.eval()
        test_loss = 0
        correct = 0
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            output_label = output.cpu().detach().numpy().round()
                        
            test_loss += criterion(output, target).data.item()
            
            for i in range(0, len(output_label)):
                print("output = ", output[i])
                print("target = ", target[i])
            
        print("accuracy = {}".format(correct))

In [147]:
test(trained_model, test_loader, criterion, device)

output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([0.9999], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([1.1418e-13], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([0.0001], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([2.0348e-14], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([0.9999], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([0.9994], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([0.9999], devic

output =  tensor([6.8332e-05], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([3.8141e-05], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([0.9992], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([0.0004], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([0.9997], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([0.0001], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([7.6643e-14], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([1.], d

output =  tensor([0.9998], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([0.0008], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([0.0001], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([0.9995], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([6.3480e-14], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([1.3338e-13], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([8.2865e-05], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([0.9998], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([0.9997], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([0.0002], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([0.0010], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =

target =  tensor(1., device='cuda:0')
output =  tensor([0.9999], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([0.0004], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([0.9996], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([0.0001], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([0.9999], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([8.8136e-14], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([0.0008], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([0.0001], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([0.0001], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([0.9997], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([1.9922e-13], device='cuda:0')
target =  te

output =  tensor([0.9996], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([9.2644e-05], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([9.4199e-05], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([4.6462e-14], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([4.2964e-14], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([5.3592e-05], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([6.5002e-05], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  te

...

output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([0.9996], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([0.9994], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([0.0002], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([0.0002], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([0.9998], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([3.4904e-05], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([0.0001], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([0.9994], devic

output =  tensor([9.2428e-05], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([1.3202e-13], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([0.9997], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([0.0002], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([0.0004], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([6.7329e-05], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([1.4998e-13], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([1.

output =  tensor([2.7637e-14], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([0.9996], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([0.0005], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([0.9995], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([0.9995], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([8.5625e-05], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([0.0001], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([0.0002], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([0.0012

target =  tensor(0., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([0.9998], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([0.0001], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([0.9995], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([0.0001], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([0.9997], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([0.0007], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([0.0001], device='cuda:0')
target =  tensor(0., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([1.], device='cuda:0')
target =  tensor(1., device='cuda:0')
output =  tensor([0.9995], device='cuda:0')
target =  tensor(1., device='cud

KeyboardInterrupt: 